# Effect of Travel on Baseball Performance
***
**Team Members:** Brandon Zink, Cameron Connor, Abiel Fattore

In this project, we will look at the effect of travel in terms of distance in miles and number of time zones changed on the traveling teams performance.

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
from math import sin, cos, sqrt, atan2, radians

In [45]:
#Read in the data, store in appropriate data frames

dfStadiumsInfo = pd.read_csv('data/Stadium_Info.csv')

#Import and clean the game logs data
dfGameInfo = pd.read_csv('data/Game_Logs.csv', low_memory=False, header=None)
#Name all of the columns
dfGameInfo.columns = (['date','gameNumber','day','awayTeam','awayTeamLg','awayTeamGmNmbr','homeTeam','homeTeamLg','homeTeamGmNmbr','awayScore','homeScore','lengthInOuts','timeOfDay','completion','forfeit','protest','parkID','attendance','lengthOfGameInMinutes','awayLineScore','homeLineScore','awayAB','awayH','away2B','away3B','awayHR','awayRBI','awaySH','awaySF','awayHBP','awayBB','awayIBB','awaySO','awaySB','awayCS','awayGIDP','awayCatchInf','awayLOB','awayPitchersUsed','awayIndivER','awayER','awayWildPitch','awayBalk','awayPO','awayAssists','awayE','awayPassedBalls','awayDoubPlay','awayTripPlay','homeAB','homeH','home2B','home3B','homeHR','homeRBI','homeSH','homeSF','homeHBP','homeBB','homeIBB','homeSO','homeSB','homeCS','homeGIDP','homeCatchInf','homeLOB','homePitchersUsed','homeIndivER','homeER','homeWildPitch','homeBalk','homePO','homeAssists','homeE','homePassedBalls','homeDoubPlay','homeTripPlay','homePlateUmpID','homePlateUmpName','1BUmpID','1BUmpName','2BUmpID','2BUmpName','3BUmpID','3BUmpName','LFUmpID','LFUmpName','RFUmpID','RFUmpName','awayManagerID','awayManagerName','homeManagerID','homeManagerName','winPitcherID','winPitcherName','losePitcherID','losePitcherName','savePitcherID','savePitcherName','GWRBIHitterID','GWRBIHitterName','awaySPID','awaySPName','homeSPID','homeSPName','away1ID','away1Name','away1POS','away2ID','away2Name','away2POS','away3ID','away3Name','away3POS','away4ID','away4Name','away4POS','away5ID','away5Name','away5POS','away6ID','away6Name','away6POS','away7ID','away7Name','away7POS','away8ID','away8Name','away8POS','away9ID','away9Name','away9POS','home1ID','home1Name','home1POS','home2ID','home2Name','home2POS','home3ID','home3Name','home3POS','home4ID','home4Name','home4POS','home5ID','home5Name','home5POS','home6ID','home6Name','home6POS','home7ID','home7Name','home7POS','home8ID','home8Name','home8POS','home9ID','home9Name','home9POS','addInfo','infoAquisition'])
#Remove the last two digits of parkID since they are useless
dfGameInfo['parkID'] = dfGameInfo['parkID'].astype(str).str[:-2].astype(str)
#Get rid of games in Tokyo, Montreal, Puerto Rico, Disney World, Sydney, Fort Bragg
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'TOK']
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'MON']
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'SJU']
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'LBV']
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'SYD']
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'FTB']

#initialize the distance matrix
dist_matrix = start_distance_matrix()

dfStadiumsInfo.head()

,teamName,teamID,lat,long,timeZone
0,Anaheim Angels,ANA,33.799572,-117.889031,0
1,Arizona Diamondbacks,ARI,33.452922,-112.038669,0
2,Atlanta Braves,ATL,33.746910,-84.391239,3
3,Baltimore Orioles,BAL,39.285243,-76.620103,3
4,Boston Red Sox,BOS,42.346613,-71.098817,3


In [4]:
#Team ID Lookup
#This is used across multiple functions to give each team a specific number that can be used in lookups (returns 0 through 29, else -1)

def team_ID_lookup(teamID):
    teamMatrix = ['ANA','ARI','ATL','BAL','BOS','CHN','CHA','CIN','CLE','COL','DET','FLO','HOU','KCA','LAN','MIL','MIN','WAS','NYN','NYA','OAK','PHI','PIT','SLN','SDN','SFN','SEA','TBA','TEX','TOR']
    for i in range (0,30):
        if(teamMatrix[i] == teamID):
            return i
    return -1

In [5]:
#Find Distance
#This function finds the distance between two sets of lat, long

def find_distance(latitude1, longitude1, latitude2, longitude2):
    # approximate radius of earth in mi
    R = 3959.0

    lat1 = radians(latitude1)
    lon1 = radians(longitude1)
    lat2 = radians(latitude2)
    lon2 = radians(longitude2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


In [21]:
#Get the time zone difference between two stadiums, returns a value between -3 and 3, where -3 means that the team traveled
#3 hours westward, and 3 means that the team traveled 3 hours eastward. Takes inputs as 3 letter code for teams. Returns
#-5 if either team is not found.

def find_time_zone_difference(awayTeam, homeTeam):
    #Convert the dataframe to an array to make it easier to loop through
    distance_array = dfStadiumsInfo.values
    
    homeTeamTZ = 1000
    awayTeamTZ = -1000
    
    #Find the time zones for the two teams
    for i in range(0,30):
        if(distance_array[i][1] == homeTeam):
            homeTeamTZ = distance_array[i][4]
        if(distance_array[i][1] == awayTeam):
            awayTeamTZ = distance_array[i][4]
    
    if(homeTeamTZ - awayTeamTZ > 3 or homeTeamTZ - awayTeamTZ < -3):
        return -5
    else:
        return homeTeamTZ - awayTeamTZ

-3

In [7]:
#Gets the distance stored in the distance matrix, returns 0 if not yet updated

def get_distance_matrix(matrix, team1ID, team2ID):
    if(matrix[team1ID][team2ID] > 0):
        return matrix[team1ID][team2ID]
    elif (matrix[team2ID][team1ID] > 0):
        return matrix[team2ID][team1ID]
    else:
        return 0

In [8]:
#Creates/resets the distance matrix to all 0, returns the matrix

def start_distance_matrix():
    return np.zeros((30, 30), dtype=float)

In [9]:
#Updates a position in the matrix to the specified distance, returns the input matrix

def update_distance_matrix(matrix, team1ID, team2ID, distance):
    matrix[team1ID][team2ID] = distance
    matrix[team2ID][team1ID] = distance
    return matrix

In [48]:
#This gets the distance between two stadiums from the stored matrix, and if it's not there, it calculates it and stores it
#in there (dist_matrix)

def get_distance(awayTeam, homeTeam):
    global dist_matrix
    
    stadium_info_array = dfStadiumsInfo.values
    distance = 0
    #get the distance between the stadiums from the distance matrix if it has already been updated
    distance = get_distance_matrix(dist_matrix, team_ID_lookup(awayTeam), team_ID_lookup(homeTeam))
    
    #if it has not been updated yet, update it
    if(distance == 0):
        #loop through the stadium matrix to find the correct row, get the lat and long from that row, calculate the distance,
        #and then update the matrix to include that distance
        awayLat = awayLong = homeLat = homeLong = -1.0
        for i in range(0,30):
            
            if(stadium_info_array[i][1] == awayTeam):
                awayLat = stadium_info_array[i][2]
                awayLong = stadium_info_array[i][3]
                
            if(stadium_info_array[i][1] == homeTeam):
                homeLat = stadium_info_array[i][2]
                homeLong = stadium_info_array[i][3]
            
        distance = find_distance(awayLat, awayLong, homeLat, homeLong)
                
        dist_matrix = update_distance_matrix(dist_matrix, team_ID_lookup(awayTeam), team_ID_lookup(homeTeam), distance)
        
    return distance

In [49]:
#turn the stadium info into an array for ease of searching
stadium_info_array = dfStadiumsInfo.values

#loop through each row (game)
for index, row in dfGameInfo.iterrows():
    #get the distance between the two parks in miles and time zones
    distance = get_distance(row['awayTeam'], row['homeTeam'])
    time_zone_diff = find_time_zone_difference(row['awayTeam'], row['homeTeam'])
    

        
        

1209.000500394519
324.3558145665407
2580.5556794342074
2434.411151203468
265.2143606094664
2437.915220196734
307.2138733203305
1324.6608122334912
2075.7515951836754
813.368779067564
846.3198546564417
2076.19109603666
1209.00050039
324.355814567
2580.55567943
1137.394812627892
2437.9152202
1324.66081223
2075.75159518
2484.271124239219
813.368779068
846.319854656
2076.19109604
1209.00050039
324.355814567
2580.55567943
2434.4111512
1137.39481263
265.214360609
2437.9152202
307.21387332
1324.66081223
2075.75159518
2484.27112424
813.368779068
846.319854656
2076.19109604
324.355814567
2580.55567943
2434.4111512
1137.39481263
265.214360609
307.21387332
1324.66081223
2484.27112424
813.368779068
846.319854656
1822.5523844611275
2133.946520854416
256.58685463590956
1713.9275330101639
1340.0501702123304
2453.6191580109735
327.3696740532098
2578.936814934432
397.4921011795752
411.38191901617176
1843.881873856604
2402.113220321963
950.0335909433804
1215.8760566571848
1822.55238446
2133.94652085
256.

1932.1984622
1391.74611085
2161.91542166
1885.90008035
265.214360609
2328.09840175
1137.39481263
324.355814567
1633.98628234
2076.19109604
603.905040027
1198.00373973
411.381919016
2151.72503612
1932.1984622
1391.74611085
2161.91542166
1633.98628234
603.905040027
1561.90432146
411.381919016
2437.9152202
1449.89558711
256.516043703
1713.92753301
1006.19402227
2453.61915801
7761.80462434
2133.94652085
807.749195539
960.471347493
1561.90432146
1011.54159257
2021.10280997
2075.75159518
2437.9152202
690.996470038
1449.89558711
256.516043703
1713.92753301
1006.19402227
2453.61915801
7761.80462434
2133.94652085
807.749195539
960.471347493
1561.90432146
1011.54159257
2021.10280997
2075.75159518
2437.9152202
690.996470038
1449.89558711
256.516043703
1713.92753301
1006.19402227
2453.61915801
7761.80462434
2133.94652085
807.749195539
960.471347493
1561.90432146
1011.54159257
2021.10280997
2075.75159518
2437.9152202
690.996470038
1460.39389404
501.230520441
1209.00050039
945.406258859
7800.3712918

256.586854636
831.026738034
88.9026854838
1002.49038346
653.253836934
327.369674053
1516.18418224
436.451626246
91.8615489357
2379.78010223
2328.12763941
460.565484349
734.574914768
256.586854636
831.026738034
88.9026854838
1002.49038346
1300.27045562
653.253836934
327.369674053
1516.18418224
180.375104215
436.451626246
91.8615489357
2379.78010223
2328.12763941
460.565484349
734.574914768
256.586854636
831.026738034
88.9026854838
1002.49038346
1300.27045562
653.253836934
327.369674053
1516.18418224
2379.78010223
2328.12763941
460.565484349
428.367180419
1086.4471105
1300.27045562
327.369674053
1341.89533226
237.278146773
406.105517206
2439.99127166
2526.31902754
869.050563305
428.367180419
299.397302167
947.069619107
1086.4471105
1371.32564506
76.7091887188
522.868759257
307.329179999
1341.89533226
237.278146773
406.105517206
2439.99127166
2526.31902754
869.050563305
428.367180419
299.397302167
947.069619107
1086.4471105
1371.32564506
76.7091887188
307.329179999
1341.89533226
237.27814

2259.94172089
911.116427445
2571.96666376
2076.19109604
1137.39481263
1932.5939105
2259.94172089
444.170658994
2571.96666376
2076.19109604
2439.99127166
547.186814986
237.278146773
1341.89533226
869.050563305
340.622841184
2526.31902754
653.253836934
734.574914768
1182.8325039
893.245100509
831.026738034
444.170658994
88.9026854838
1559.99730585
2439.99127166
547.186814986
237.278146773
1341.89533226
869.050563305
340.622841184
2526.31902754
653.253836934
734.574914768
1182.8325039
893.245100509
831.026738034
444.170658994
88.9026854838
1559.99730585
2439.99127166
547.186814986
237.278146773
1341.89533226
869.050563305
340.622841184
2526.31902754
653.253836934
734.574914768
1182.8325039
893.245100509
831.026738034
88.9026854838
1559.99730585
2328.12763941
180.375104215
307.851572322
460.565484349
1516.18418224
2379.78010223
205.825094725
593.837522534
256.516043703
1586.41421944
1006.19402227
1086.4471105
5429.80920875
299.397302167
299.397302167
947.069619107
2328.12763941
180.3751042

1573.55522215
7462.88909569
585.8685543601056
265.214360609
2027.5714663849078
1519.5805548924698
1143.8894397359156
1889.1842773353103
2588.5775292597514
1732.7659514347765
1102.2948062797914
982.9787275486418
1083.8333441611794
2429.801564679106
2449.467557481921
7462.88909569
585.86855436
2027.57146638
1519.58055489
1143.88943974
1889.18427734
2588.57752926
1732.76595143
1102.29480628
982.978727549
1083.83334416
2429.80156468
2449.46755748
7462.88909569
585.86855436
265.214360609
2027.57146638
1519.58055489
1143.88943974
1889.18427734
2588.57752926
1732.76595143
1102.29480628
982.978727549
1083.83334416
2429.80156468
2449.46755748
356.096043037
680.818555241
1202.19468825
1202.19468825
256.586854636
1010.5104604
831.026738034
754.713036819
456.208261975
177.434188839
547.186814986
643.102320507
356.096043037
680.818555241
1111.66598372
1202.19468825
256.586854636
1010.5104604
1012.29277576
831.026738034
754.713036819
5687.82822932
456.208261975
327.369674053
177.434188839
547.186814

...

598.628598512
1090.36281453
1932.5939105
1836.85358474
88.9026854838
5687.82822932
1268.63239029
84.2147266585
1250.14470534
1038.77994014
1118.93121177
1097.70461345
2075.75159518
2526.31902754
436.451626246
598.628598512
1090.36281453
1932.5939105
1836.85358474
88.9026854838
5687.82822932
1268.63239029
1968.84534914
603.905040027
603.905040027
2557.45579002
2021.10280997
1561.90432146
2110.78357886
2133.94652085
1968.84534914
867.672822603
938.965405206
690.996470038
2557.45579002
2021.10280997
1561.90432146
413.3570752
402.902682808
1460.39389404
346.922843863
1573.50550018
2110.78357886
2133.94652085
1586.41421944
1968.84534914
867.672822603
938.965405206
690.996470038
2557.45579002
2021.10280997
1561.90432146
402.902682808
1460.39389404
346.922843863
1573.50550018
2110.78357886
2133.94652085
1586.41421944
1968.84534914
867.672822603
938.965405206
690.996470038
2557.45579002
2021.10280997
1561.90432146
413.3570752
402.902682808
1460.39389404
346.922843863
1573.50550018
2110.7835788

665.851498066
832.244086134
265.214360609
212.631100449
91.8615489357
1669.34762928
754.713036819
307.329179999
444.170658994
832.244086134
1348.9495484
960.471347493
356.096043037
340.622841184
1019.67526326
1198.00373973
1453.89000231
359.127072414
754.713036819
307.329179999
352.114535329
665.803540558
665.803540558
444.170658994
456.208261975
1348.9495484
960.471347493
356.096043037
340.622841184
340.622841184
1019.67526326
1198.00373973
1453.89000231
359.127072414
754.713036819
307.329179999
352.114535329
665.803540558
444.170658994
456.208261975
1348.9495484
960.471347493
356.096043037
91.8615489357
91.8615489357
340.622841184
1019.67526326
1198.00373973
1453.89000231
359.127072414
754.713036819
307.329179999
352.114535329
665.803540558
444.170658994
456.208261975
357.798185727
1843.88187386
698.725822949
540.790766333
981.234182881
1028.70368629
229.976850524
299.397302167
831.026738034
346.922843863
123.776266595
2037.26099455
357.798185727
1843.88187386
698.725822949
540.79076

1038.77994014
1040.60265556
1571.71706782
1061.35453923
340.622841184
1012.29277576
2133.94652085
1741.24954898
1090.36281453
5687.82822932
327.369674053
719.079334692
123.776266595
1968.84534914
357.798185727
1038.77994014
1012.29277576
1040.60265556
1571.71706782
340.622841184
2133.94652085
1741.24954898
1090.36281453
5687.82822932
327.369674053
719.079334692
123.776266595
1061.35453923
357.798185727
698.725822949
483.249413864
1012.29277576
1324.66081223
2133.94652085
1741.24954898
1090.36281453
5687.82822932
692.37397657
1061.35453923
177.434188839
428.367180419
698.725822949
237.278146773
1630.12976587
1324.66081223
1669.34762928
1449.89558711
402.902682808
1932.5939105
346.922843863
692.37397657
2119.91005196
832.244086134
1740.43538159
177.434188839
428.367180419
698.725822949
237.278146773
1630.12976587
1324.66081223
1669.34762928
1449.89558711
402.902682808
1932.5939105
346.922843863
692.37397657
2119.91005196
832.244086134
1740.43538159
177.434188839
428.367180419
698.7258229

663.672240196
299.397302167
357.798185727
698.725822949
1630.12976587
2066.16613729
1028.70368629
1202.19468825
256.586854636
794.2518957
345.404208043
352.114535329
444.170658994
754.713036819
663.672240196
299.397302167
357.798185727
698.725822949
1348.9495484
540.790766333
540.790766333
1028.70368629
831.026738034
734.574914768
444.170658994
88.9026854838
456.208261975
1998.14227777
307.851572322
643.102320507
1348.9495484
1391.62141634
1453.89000231
1198.00373973
340.622841184
265.214360609
831.026738034
734.574914768
324.355814567
88.9026854838
193.727005798
456.208261975
1998.14227777
307.851572322
643.102320507
1348.9495484
1391.62141634
1453.89000231
1198.00373973
340.622841184
265.214360609
831.026738034
734.574914768
324.355814567
88.9026854838
193.727005798
456.208261975
1998.14227777
307.851572322
643.102320507
1348.9495484
1391.62141634
1453.89000231
1198.00373973
340.622841184
265.214360609
831.026738034
324.355814567
88.9026854838
193.727005798
456.208261975
364.31347662

1586.51518715
402.902682808
7800.37129184
692.37397657
1633.98628234
556.601637571
456.208261975
2122.38888359
606.865107752
180.375104215
1734.10183672
205.825094725
1057.50795014
460.565484349
2151.72503612
1586.51518715
402.902682808
7800.37129184
692.37397657
1633.98628234
556.601637571
456.208261975
2122.38888359
606.865107752
180.375104215
1734.10183672
205.825094725
1057.50795014
460.565484349
2151.72503612
1586.51518715
402.902682808
7800.37129184
692.37397657
1633.98628234
556.601637571
456.208261975
2122.38888359
606.865107752
180.375104215
180.375104215
1734.10183672
205.825094725
1057.50795014
460.565484349
2151.72503612
593.837522534
402.902682808
832.244086134
5429.80920875
880.038718702
444.170658994
2160.94065484
1225.99787479
428.367180419
1889.18427734
643.102320507
1016.88438133
2379.78010223
593.837522534
1575.85029998
832.244086134
5429.80920875
880.038718702
444.170658994
2160.94065484
2283.03725612
1225.99787479
428.367180419
1741.22283904
1889.18427734
643.10232

In [50]:
#This just checks the distance matrix to make sure it is updating correctly

def check_distance_matrix():
    for i in range(0,30):
        print(dist_matrix[i])
        
check_distance_matrix()

[    0.           337.4076805   1915.35442255  2301.96609828  2578.93681493
  1731.78669214  1731.63366243  1880.16136546  2034.56810013   824.02808677
  1968.84534914  2306.48287509  1348.9495484   1341.89533226    27.86270781
  1732.76595143  1516.18418224  2283.03725612  2442.68219594  2437.9152202
   364.31347663  2376.52958258  2119.91005196  1573.55522215    84.21472666
   373.0973964    981.23418288  2122.38888359  1202.19468825  2161.91542166]
[  337.4076805      0.          1586.51518715  1998.14227777  2291.29344504
  1449.89558711  1448.7742975   1575.85029998  1744.17715533   585.83387096
  1685.14111247  1969.12561801  1012.29277576  1045.55797299   359.12707241
  1460.39389404  1276.97359739  1977.1265442   2146.54763217  2142.10882191
   642.76033716  2076.19109604  1822.55238446  1268.63239029   299.39730217
   653.25383693  1114.85597174  1785.01769687   866.40577638  7462.88909569]
[ 1915.35442255  1586.51518715     0.           576.3702812    934.12110322
   593.8375